In [5]:
import pandas as pd
import numpy as np

In [6]:
tweets = pd.read_csv('tweets.csv')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6296495 entries, 0 to 6296494
Data columns (total 18 columns):
Unnamed: 0             int64
id                     float64
text                   object
user_id                float64
in_reply_to_user_id    object
retweeted_status_id    object
retweet_count          float64
reply_count            float64
favorite_count         float64
num_hashtags           float64
num_urls               float64
num_mentions           float64
created_at             object
timestamp              object
crawled_at             object
updated                object
Cat                    object
bot                    object
dtypes: float64(8), int64(1), object(9)
memory usage: 864.7+ MB


In [13]:
#tweets_humans = tweets[tweets.Cat=='Humans']
#display(tweets_humans.columns)

#first = True
def getDataTweets(twU):
    global first
    if(first):
        dates = pd.to_datetime(twU.timestamp) #aca lo que hago es guardar la columnoa como datetime
        twU = twU.sort_values(by='timestamp', ascending=0)
        twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
        twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
        twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))
        #display(twU.head())
        twU.loc[:,'interval'] = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp)
        times = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp) # calculo distancia entre tweets
        #display(times.median())
        
        #ver word_to_vect
        
        print(len(twU.weekNum.unique()))
        
        return [times.median()]FFF
        #time_median, time_std, max_cuart, min_cuart, media_lun, media_martes, dispercion_horario
        #media_palabras
        #cantidad de palabras unicas
        
        #url, hashtags, menciones, reply, rt,  favs
        
        #cantidad_usr que retwittea / cantidad de retweets
        
    first = False

#tweets_humans.groupby('user_id').apply(getDataTweets).hour

first = True
tweets_humans = tweets[tweets.Cat=='Humans']
a = tweets_humans[['user_id','timestamp']]
b = a.groupby('user_id').apply(getDataTweets)
#b.groupby('user_id').mean()
b

27
27
89
30
91
23
8
59
97
79
38
22
18
165
11
16
115
83
19
52
267
20
3
217
65
242
55
30
30
5
52
31
298
84
76
229
103
124
13
79
131
35
9
129
11
82
120
8
31
24
133
30
15
125
180
115
295
171
251
257
21
126
29
88
84
73
34
16
160
5
316
15
25
147
68
22
155
31
308
177
176
264
4
42
250
11
57
82
41
54
39
93
105
9
121
96
116
224
135
5
5
175
179
206
111
117
55
137
298
61
321
91
120
74
8
18
104
4
41
19
90
55
35
178
33
210
80
35
51
50
23
57
117
96
216
29
164
116
18
10
325
45
325
18
178
70
214
14
91
54
11
29
128
157
10
108
95
131
37
54
165
258
149
7
208
88
43
8
256
185
261
193
257
202
136
72
34
56
131
107
320
215
250
155
26
21
8
79
173
68
51
93
248
32
89
128
123
242
158
116
27
38
155
51
102
63
33
66
107
36
261
9
43
104
78
103
47
4
203
224
236
31
67
85
7
107
82
211
25
14
10
109
92
73
101
198
203
108
62
67
147
11
291
96
14
16
118
107
53
90
10
89
166
10
18
131
135
144
10
85
102
113
35
64
167
60
188
124
159
83
21
146
79
11
59
93
6
17
54
37
75
53
58
40
32
20
10
91
42
25
89
13
17
5
122
95
66
162
7
35
179
2

user_id
6.780330e+05    [0 days 00:05:11.500000]
7.226230e+05           [0 days 01:31:45]
7.551160e+05           [0 days 00:06:09]
7.557460e+05           [0 days 01:36:58]
7.850800e+05           [0 days 00:01:27]
8.065850e+05           [0 days 00:02:16]
8.069750e+05           [0 days 00:04:52]
8.872810e+05           [0 days 01:00:43]
1.382561e+06           [0 days 00:11:20]
1.813221e+06           [0 days 00:03:56]
2.770511e+06           [0 days 00:07:25]
3.873101e+06           [0 days 00:06:27]
3.888491e+06    [0 days 01:06:05.500000]
4.121091e+06           [0 days 00:04:30]
5.445622e+06           [0 days 00:05:40]
5.812422e+06           [0 days 00:52:59]
5.820222e+06           [0 days 00:41:29]
5.856182e+06    [0 days 00:06:16.500000]
6.296742e+06           [0 days 00:14:09]
6.775342e+06           [0 days 11:21:36]
6.834152e+06           [0 days 00:00:45]
7.004532e+06           [0 days 00:01:37]
8.548002e+06           [0 days 01:16:49]
8.657052e+06           [0 days 00:30:47]
8.711952

27
27
27
27


NameError: name 'times' is not defined

In [41]:
b[['user_id', 'dayOfWeek', 'hour', 'weekNum', 'interval']].groupby('user_id').max()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'user_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


,dayOfWeek,hour,weekNum,interval
user_id,,,,
6.780330e+05,6,23,9-2015,1 days 11:52:06
7.226230e+05,6,23,9-2015,3 days 01:29:20
7.551160e+05,6,23,9-2015,2 days 21:45:47
7.557460e+05,6,23,9-2015,2 days 16:44:34
7.850800e+05,6,23,9-2015,2 days 11:41:22
8.065850e+05,6,23,18-2015,0 days 15:46:25
8.069750e+05,6,23,9-2015,9 days 17:27:33
8.872810e+05,6,23,9-2015,220 days 19:18:13
1.382561e+06,6,23,9-2015,36 days 01:58:12
